In [0]:
import warnings
warnings.filterwarnings("ignore")

In [0]:
from datetime import datetime

import numpy as np
import torch
import pandas as pd

import random
import collections

from progressbar import ProgressBar   

# DATA:

In [0]:
path = './data/'

In [0]:
%time df = pd.read_csv(path +'data.csv')
df.columns = 'movie user rating date'.split()

df.head()

Wall time: 45.4 s


,movie,user,rating,date
0,1,822109,5,2005-05-13
1,1,885013,4,2005-10-19
2,1,30878,4,2005-12-26
3,1,823519,3,2004-05-03
4,1,893988,3,2005-11-17


In [0]:
df = df.iloc[:, :-1]
df.head()

,movie,user,rating
0,1,822109,5
1,1,885013,4
2,1,30878,4
3,1,823519,3
4,1,893988,3


In [0]:
listO_movies = df.movie.unique()
n_movies = len(listO_movies)
n_movies

17770

In [0]:
listO_users = df.user.unique()
n_users = len(listO_users)
n_users

480189

In [0]:
n_ratings = len(df)
n_ratings

100480506

# TRAIN & TEST SET FORMATTING:

### 1. Sample 5000 movies:

In [0]:
movies = list(range(1, n_movies+1))
n_samples = 5000

sampled_movies = np.random.choice(movies, n_samples, replace=False)
np.save(path + '5000_sampled_movies', sampled_movies)

In [0]:
sampled_movies = np.load(path + '5000_sampled_movies.npy')
sampled_movies

array([10423,  5477,  5658, ...,  5726, 10391,  6313])

In [0]:
def retrn_dataset_sampled_by_movies(df, sampled_movies):
    
    from progressbar import ProgressBar
    pbar = ProgressBar()
    
    def retrn_tensorO_movie(movie):
        matrixO_movies = df[df.movie == movie].values
        tensorO_movie = torch.LongTensor(matrixO_movies)
        return tensorO_movie
    
    
    tensorO_data = torch.LongTensor()
    anamoly = []
    
    sampled_movies = pbar(sampled_movies)
    for movie in sampled_movies:
        
        try:
            tensorO_movie = retrn_tensorO_movie(movie)
        except:
            anamoly.append(user)
            
        tensorO_data = torch.cat((tensorO_data, tensorO_movie)) 
    
    return tensorO_data

In [0]:
sampled_movies = sampled_movies

tensorO_data_sampled_movies = retrn_dataset_sampled_by_movies(df, sampled_movies)

100% (5000 of 5000) |####################| Elapsed Time: 0:29:00 Time:  0:29:00


In [0]:
tensorO_data_sampled_movies

tensor([[  10423, 1527030,       4],
        [  10423,  306466,       3],
        [  10423,  101470,       4],
        ...,
        [   6313,  584661,       4],
        [   6313, 1333111,       3],
        [   6313, 1055707,       4]])

In [0]:
len(tensorO_data_sampled_movies[:, 0].unique())

5000

In [0]:
len(tensorO_data_sampled_movies[:, 1].unique())

475790

In [0]:
tensorO_data_sampled_movies.shape

torch.Size([28136109, 3])

In [0]:
np.save(path + 'tensorO_data_sampled_movies', tensorO_data_sampled_movies)

***

### 2. Sample 50,000 users from dataset created above:

In [0]:
users = tensorO_data_sampled_movies[:, 1].unique()
n_samples = 50000
sampled_users = np.random.choice(users, n_samples, replace=False)
np.save(path + '50000_sampled_users', sampled_users)

In [0]:
sampled_users = np.load(path + '50000_sampled_users.npy')
sampled_users

array([2638384, 1279309, 1272122, ..., 2112276, 1648497, 2609544],
      dtype=int64)

In [0]:
sampled_users.shape

(50000,)

In [0]:
df_sampled_movies = pd.DataFrame(tensorO_data_sampled_movies.numpy(), columns = 'movie user rating'.split())
print(df_sampled_movies.shape)

df_sampled_movies.head()

(28136109, 3)


,movie,user,rating
0,10423,1527030,4
1,10423,306466,3
2,10423,101470,4
3,10423,1945809,4
4,10423,464539,5


In [0]:
def retrn_dataset_sampled_by_users(df, sampled_users):
    
    from progressbar import ProgressBar
    pbar = ProgressBar()
    
    def retrn_tensorO_user(user):
    
        matrixO_user = df[df.user == user].values
        tensorO_user = torch.LongTensor(matrixO_user)
        return tensorO_user
    
    
    tensorO_data = torch.LongTensor()
    anamoly = []
    
    sampled_users = pbar(sampled_users)
    for user in sampled_users:
        
        try:
            tensorO_user = retrn_tensorO_user(user)
        except:
            anamoly.append(user)
            
        tensorO_data = torch.cat((tensorO_data, tensorO_user)) 
    
    return tensorO_data

In [0]:
df = df_sampled_movies
sampled_users = sampled_users

sampled_dataset = retrn_dataset_sampled_by_users(df, sampled_users)

100% (50000 of 50000) |##################| Elapsed Time: 1:14:52 Time:  1:14:52


In [0]:
sampled_dataset

tensor([[    367, 2638384,       5],
        [   6366, 2638384,       3],
        [   3444, 2638384,       4],
        ...,
        [   3870, 2609544,       3],
        [  15997, 2609544,       4],
        [  17088, 2609544,       3]])

In [0]:
sampled_dataset.shape

torch.Size([2938010, 3])

In [0]:
len(sampled_dataset[:, 0].unique()), len(sampled_dataset[:, 1].unique())

(4998, 50000)

In [0]:
np.save(path + 'sampled_dataset', sampled_dataset)

***

### 3. TRAIN-TEST SPLIT:

In [0]:
df_sampled_dataset = pd.DataFrame(sampled_dataset.numpy(), columns = 'movie user rating'.split())
print(df_sampled_dataset.shape)

df_sampled_dataset.head()

(2938010, 3)


,movie,user,rating
0,367,2638384,5
1,6366,2638384,3
2,3444,2638384,4
3,11452,2638384,4
4,12999,2638384,5


In [0]:
def shuffle(df):
    index = list(df.index)
    random.shuffle(index)
    df = df.ix[index]
    df.reset_index()
    return df

In [0]:
df_sampled_dataset = shuffle(df_sampled_dataset)
df_sampled_dataset.head()

,movie,user,rating
333064,8312,2010587,3
911833,11040,2143980,5
1287538,6231,379605,3
941269,15843,1213511,3
1694504,3626,2232792,4


In [0]:
limit = int(0.8 * len(sampled_dataset))

df_trainset = df_sampled_dataset.iloc[:limit, :]
df_testset  = df_sampled_dataset.iloc[limit:, :]

df_trainset.shape, df_testset.shape

((2350408, 3), (587602, 3))

In [0]:
df_trainset.head()

,movie,user,rating
333064,8312,2010587,3
911833,11040,2143980,5
1287538,6231,379605,3
941269,15843,1213511,3
1694504,3626,2232792,4


In [0]:
df_testset.head()

,movie,user,rating
628153,10055,19143,4
170300,8387,1360578,3
1499509,14718,1650184,1
588472,3538,159056,4
2513615,3936,1150537,2


### 4. Filter out those users from testset which are not preset in trainset:
- AVOID COLD START

In [0]:
unique_train_users, unique_test_users = df_trainset.iloc[:, 1].unique(), df_testset.iloc[:, 1].unique()

In [0]:
unique_train_users.shape, unique_test_users.shape

((49800,), (45565,))

In [0]:
new_users = list(set(unique_test_users) - set(unique_train_users))
len(new_users)

200

In [0]:
def fn_remove_new_users(df_testset, new_users):
    
    pbar = ProgressBar()
    
    for new_user in pbar(new_users):
        df_testset = df_testset[df_testset.user != new_user]
        
    return df_testset

In [0]:
df_testset = fn_remove_new_users(df_testset, new_users)
df_testset.head()

100% (200 of 200) |######################| Elapsed Time: 0:00:04 Time:  0:00:04


,movie,user,rating
628153,10055,19143,4
170300,8387,1360578,3
1499509,14718,1650184,1
588472,3538,159056,4
2513615,3936,1150537,2


In [0]:
df_testset.shape

(587329, 3)

### CHECKING IF TESTSET CONTAINS ANY 'NEW' USERS :

In [0]:
unique_train_users  = df_trainset.iloc[:, 1].unique() 
unique_test_users = df_testset.iloc[:, 1].unique()

new_users = list(set(unique_test_users) - set(unique_train_users))
len(new_users)

0

***

### 5. Embed users & movies as continuous numbers:
- Pytorch Embedding layer expects numerical IDs to lie withing the range of the Embedding dimension size specified
- ie: If n_users = 5000, then all unique user ids should have values between (0, 5000)

In [0]:
def fn_embed_id_cols(df_train_set, df_test_set):
    
    pbar = ProgressBar()
    
    df_trainset = df_train_set.copy()
    df_testset  = df_test_set.copy()
    
    columns = pbar(df_trainset.columns[:-1])
    for col in columns:
        unique = df_trainset[col].unique()
        
        look_up_table = {j : i for i, j in enumerate(unique)}
        
        trainset_col = [look_up_table[j] for j in df_trainset[col].values]
        testset_col  = [look_up_table[j] for j in df_testset[col].values]
        
        df_trainset[col] = trainset_col
        df_testset[col]  = testset_col
        
    return df_trainset, df_testset


In [0]:
df_trainset_encoded, df_testset_encoded = fn_embed_id_cols(df_trainset, df_testset)

100% (2 of 2) |##########################| Elapsed Time: 0:00:02 Time:  0:00:02


In [0]:
df_trainset_encoded.head()

,movie,user,rating
333064,0,0,3
911833,1,1,5
1287538,2,2,3
941269,3,3,3
1694504,4,4,4


In [0]:
df_testset_encoded.head()

,movie,user,rating
628153,161,11581,4
170300,347,5268,3
1499509,67,13098,1
588472,529,4808,4
2513615,272,7473,2


In [0]:
df_trainset_encoded.shape, df_testset_encoded.shape

((2350408, 3), (587329, 3))

### CHECKING IF TESTSET CONTAINS ANY 'NEW' USERS :

In [0]:
set(df_testset_encoded.user.unique()) - set(df_trainset_encoded.user.unique())

set()

### SAVE:

In [0]:
df_trainset_encoded.to_csv(path + 'df_final_trainset.csv')
df_testset_encoded.to_csv(path + 'df_final_testset.csv')

***